# Homework 2: Word Similarity

Student Name: Xinnan SHEN

Student ID: 1051380

## General info

<b>Due date</b>: Thursday, 4 June 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

**Note**: As we will be implementing neural networks in this assignment, you're encouraged to build your notebook on **colab**. See the programming exercise in workshop-07 (`10-bert.ipynb`) if you are not familiar with colab.

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.




<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dataset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

### Question 1 (1.0 mark)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://alfonseca.org/eng/research/wordsim353.html">Similarity-353</a>. You need to first obtain this dataset, which is on Canvas (assignment 2). The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on **document frequencies** (not token frequencies) in the Brown corpus, in order to remove rare words. In this homework, **we will be treating the paragraphs of the Brown corpus as our "documents"**. You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized (lemmatize function provided). Store this preprocessed data in *brown_corpus* object (it will be used for question 4 and 5 later).

Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of **$< 8$** in this corpus. You should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*.

Note: the document frequency of a word denotes the number of documents that contains the word.

**Task**: Filter word pairs from *set1.tab* based on document frequencies. Produce *brown_corpus*, which is a list where each element is a set of words for one paragraph (e.g. the first element in *brown_corpus* should contain all the unique word types for the first paragraph). Produce *filtered_gold_standard*, a dictionary of filtered word pairs with human similarity ratings (the dictionary should have (word1, word2) as keys, and similarity ratings as values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *filtered_gold_standard*.

In [37]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

# lemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


###
# Your answer BEGINS HERE
###
def search(word,corpus):
    sum=0
    for doc in corpus:
        if word in doc:
            sum=sum+1
        if sum>=8:
            break
    return sum


#read dataset into a dictionary
file=open("set1.tab","r")
strlist=file.readlines()
file.close()
raw_dict={}
for ele in strlist:
    if strlist.index(ele)==0:
        continue
    temp_str=ele.split("\t")[0:3]
    raw_dict[(temp_str[0],temp_str[1])]=float(temp_str[2])

#generate brown_corpus
brown_raw=brown.paras()
brown_corpus=[]
for doc in brown_raw:
    doc_temp=[]
    for sen in doc:
        sen_temp=[]
        for word in sen:
            if word.isalpha():
                sen_temp.append(lemmatize(word.lower()))
        doc_temp.extend(sen_temp)
    brown_corpus.append(doc_temp)
#filter dictionary
filtered_gold_standard=raw_dict
for key in list(filtered_gold_standard.keys()):
    word1=key[0]
    word2=key[1]
    result1=search(word1,brown_corpus)
    result2=search(word2,brown_corpus)
    
    if result1<8 or result2<8:
        filtered_gold_standard.pop(key)
###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'):

<b>For your testing: </b>

In [0]:
assert(len(brown_corpus)==15667)
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

### Question 2 (1.0 mark)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a **single primary sense**. We define single primary sense here as either: (a) having only one sense (i.e. only one synset), or (b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma.

Note: You should lowercase the lemmas of a synset when matching your word; and if there are multiple lowercased lemmas that match your word, you should sum up the count of all matching lemmas.

Additionally, you should remove any words where the primary sense is **not a noun** (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section.

Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria. When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*.

**Task**: Filter word pairs for any words which do not have a single primary sense and aren't nouns. Produce *final_gold_standard*, a dictionary of filtered word pairs with human similarity ratings. Note: this second filtering is applied on top of the first filtering (question 1). In other words, you shouldn't consider any word pairs that have already been discarded by the first filtering in this question.

**Check**: Use the assertion statements in *"For your testing"* for the expected *final_gold_standard*.

In [39]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}
word_primarysense = {} #a dictionary of (word, primary_sense) (used for next section); primary_sense is a synset

###
# Your answer BEGINS HERE
###
def freq(synset,word):
    lemmas=synset.lemmas()
    count=0
    for l in lemmas:
        if l.name().lower()==word.lower():
            count=count+l.count()
    return count

def haveSinglePrimarySense(word):
    synsets=wordnet.synsets(word)
    
    #find the primary and secondary senses
    if len(synsets)==1:
        word_primarysense[word]=synsets[0]
        return True
    for i in range(0,len(synsets)):
        for j in range(i+1,len(synsets)):
            fr1=freq(synsets[i],word)
            fr2=freq(synsets[j],word)
            if fr1<fr2:
                temp=synsets[i]
                synsets[i]=synsets[j]
                synsets[j]=temp
    primarysense=synsets[0]
    secondarysense=synsets[1]            
    
    #judge whether the word is noun
    pos=primarysense.pos()
    
    if pos!='n':
        return False
    word_primarysense[word]=primarysense
    lemmas0=primarysense.lemmas()
    lemmas1=secondarysense.lemmas()
    #calculate the count of primary sense
    sum_count0=0
    for l0 in lemmas0:
        lemma=l0.name()
        if lemma.lower()==word.lower():
            count=l0.count()
            sum_count0=sum_count0+count
    #calculate the count of secondary sense
    sum_count1=0
    for l1 in lemmas1:
        lemma=l1.name()
        if lemma.lower()==word.lower():
            count=l1.count()
            sum_count1=sum_count1+count
    
    if sum_count0==0:
        return False
    else:
        return sum_count0>=4*sum_count1

#filter gold standard
for key,value in filtered_gold_standard.items():
    #test if both of them have single primary sense
    if haveSinglePrimarySense(key[0]) and haveSinglePrimarySense(key[1]):
        final_gold_standard[key]=value

###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [0]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Computing word similiarity with Lin similarity, NPMI and LSA (3 marks)

### Question 3 (1.0 mark)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*.

**Task**: Compute word pair similarity using Lin similarity for the test set. Produce *lin_similarities*, a dictionary of word pairs (keys) and computed Lin similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *lin_similarities*. 

In [41]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###
#load information content in brown corpus
ic=wordnet_ic.ic('ic-brown.dat')
#iterate over the final gold standard, and calculate Lin similarities
for key in list(final_gold_standard.keys()):
    #get the first and second word senses
    sense0=word_primarysense[key[0]]
    sense1=word_primarysense[key[1]]
    #calculate Lin similarity
    lin=sense0.lin_similarity(sense1,ic)
    #store result
    lin_similarities[key]=lin


###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psychology', 'health'): 0.0600

<b>For your testing:</b>

In [0]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

### Question 4 (1.0 mark)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-log_2(p(x,y))} = \frac{log_2(p(x)p(y))}{log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1]. 

You should use the *brown_corpus* object you've created in question 1 as your corpus here. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*.

**Task**: Compute word pair similarity using NPMI similarity for the test set. Produce *NPMI_similarities*, a dictionary of word pairs (keys) and computed NPMI similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *NPMI_similarities*.

In [44]:
import math
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###

#calculate the total number of word occurences
def caltotal():
    #wordlist stores all words in brown_corpus
    wordlist=[]
    for doc in brown_corpus:
        for token in doc:
            if token not in wordlist:
                wordlist.append(token)
    #the function returns the length of wordlist, which is the number of total word occurences
    return len(wordlist)

#calculate the number of occurences of a given word
def caloccur(word):
    count=0
    for doc in brown_corpus:
        #if the word occurs in the document, increase count
        if word in doc:
            count=count+1
            
    return count


#calculate the number of cooccurences of given two words
def calcooccur(word1,word2):
    count=0
    for doc in brown_corpus:
        #if both words occur in the same document, then increase count
        if (word1 in doc) and (word2 in doc):
            count=count+1
    return count
#get the total number of word occurences
N=caltotal()
#get the probability of word cooccurences
def pxy(x,y):
    return calcooccur(x,y)/N
#get the probability of word occurences
def p(word):
    return caloccur(word)/N
#get the final NPMI value of given two words
def npmi(x,y):
    p_cooccur=pxy(x,y)
    if p_cooccur==0:
        return -1
    else:
        return math.log(p(x)*p(y),2)/math.log(p_cooccur,2)-1
#iterate the final gold standard, and store the NPMI result
for key in list(final_gold_standard.keys()):
    NPMI_similarities[key]=npmi(key[0],key[1])

###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.5345151842582327, ('professor', 'doctor'): -1, ('student', 'professor'): 0.35506998677905255, ('stock', 'egg'): 0.17565558576760232, ('money', 'cash'): 0.255136678067456, ('king', 'queen'): 0.2404398547154747, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.6191546407863497, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.5100013701085555, ('baby', 'mother'): 0.3322049774484772, ('car', 'automobile'): 0.3631907793760183, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.4362988887339576, ('brother', 'monk'): 0.25583200784673155, ('journey', 'car'): -1, ('coast', 'hill'): 0.13753445378066775, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.28033635534315904, ('psychology', 'doctor'): 0.29681680230681917, ('psychology', 'mind'): 0.26479501562071417, ('psychology', 'health'): -1, ('psychology', 'science'): 0.4509746030184476, ('planet', 'moon'): 0.5341647266938854, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [0]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

### Question 5 (1.0 mark)

**Instructions:** Here we'll use singular value decomposition (SVD) to derive similarity scores using the Latent Semantic Analysis (LSA) method. Recall that LSA applies SVD and truncation to get a dense vector representation of a word type. To measure similarity between two words we calculate cosine similarity between the LSA vectors of the words.

We'll first build a term-document frequency matrix (as before, a document is a paragraph in Brown corpus). That is, the rows corresponds to words in the vocabulary, and the columns represent the paragraphs/documents in Brown corpus. Each cell records the document frequency of a word (1 if the word appears in the document, 0 otherwise). You should use the *brown_corpus* object created in question 1 as your corpus to build the term-document frequency matrix.

Given the term-document frequency matrix, we'll use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for the word pairs in the test set.

You can use the workshop's notebook on distributional similarity (``09-distributional-similarity.ipynb``) as a starting point, but note that we use term-document frequency matrix (as opposed to the tf-idf matrix in the workshop).

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*.

**Task**: Compute word pair similarity using LSA. Produce *LSA_similarities*, a dictionary of word pairs (keys) and computed LSA similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *LSA_similarities*. 



In [48]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np
from numpy.linalg import norm
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###

#return a term-frequency dictionary for a given document
def gettf_dict(doc):
    tf_dict={}
    for word in doc:
        tf_dict[word]=1
    return tf_dict
#calculate the cosine similarity of two words
def cos_sim(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

list_=[]
for doc in brown_corpus:
    list_.append(gettf_dict(doc))
#produce dense vectors
vectorizer = DictVectorizer()
matrix=vectorizer.fit_transform(list_)
matrix_transposed=csr_matrix(matrix).transpose()
svd=TruncatedSVD(n_components=500)
newmatrix=svd.fit_transform(matrix_transposed)
#iterate over final gold standard, calculate and store the LSA similarities
for key in list(final_gold_standard.keys()):
    v1=newmatrix[vectorizer.vocabulary_[key[0]]]
    v2=newmatrix[vectorizer.vocabulary_[key[1]]]
    LSA_similarities[(key[0],key[1])]=cos_sim(v1,v2)
###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.3066013916675333, ('professor', 'doctor'): 0.04216174334737762, ('student', 'professor'): 0.28812390586380254, ('stock', 'egg'): 0.10838154002348296, ('money', 'cash'): 0.091898144988949, ('king', 'queen'): 0.09135403102883734, ('bishop', 'rabbi'): 0.03981066488050217, ('football', 'basketball'): 0.2751932178854581, ('football', 'tennis'): 0.15968431251967835, ('alcohol', 'chemistry'): 0.09913541330587054, ('baby', 'mother'): 0.34931534842714196, ('car', 'automobile'): 0.3459890233444121, ('journey', 'voyage'): 0.07959677694422683, ('coast', 'shore'): 0.43434943558522815, ('brother', 'monk'): 0.05171394832523009, ('journey', 'car'): -0.013965150887771074, ('coast', 'hill'): 0.21202295295396067, ('forest', 'graveyard'): 0.06328330404583372, ('monk', 'slave'): -0.028972745607760267, ('coast', 'forest'): 0.12033702166706951, ('psychology', 'doctor'): 0.11648070031140519, ('psychology', 'mind'): 0.10632384808105998, ('psychology', 'health'): 0.049515728943221225, ('

<b>For your testing:</b>

In [0]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## Computing word similarity with feedforward language model (3 marks)

### Question 6 (1.0 mark)

**Instructions**: Here we'll build a n-gram neural language model to learn word embeddings, and compute word similarity based on the word embeddings. As before we will use the Brown corpus as training data for our language model, and the first step is to collect the vocabulary.

As before, we'll treat paragraphs in the Brown corpus as our documents. The first 12K paragraphs/documents will serve as our training data, and the rest (3K+ documents) as development data. The first step towards building a language model is to collect the vocabulary, i.e. the set of unique word types in our training data. When collecting the word types, you should lowercase all words, and only keep word types that have a frequency $>= 5$. Store your vocabulary in the _vocab_ object.

Note: 
  - we'll be using _$<$UNK$>$_ to represent unseen words, and so _vocab_ is initialised with the special _$<$UNK$>$_.
  - you do not need to do any other additional preprocessing aside from the aforementioned steps (e.g. no need to remove symbols, etc)
  - you should not use *brown_corpus* here, as you need the words in their original order (*brown_corpus* stores a set of words for each paragraph, and so do not contain word order information)

**Task**: Collect a set of unique word types in the training portion (first 12K paragraphs) of the Brown corpus. Produce _vocab_, which is a set that contains all the word types.

**Check**: Use the assertion statements in *"For your testing"* below to check the vocabulary size.

In [50]:
num_train = 12000
UNK_symbol = "<UNK>"
vocab = set([UNK_symbol])

###
# Your answer BEGINS HERE
###
v=list(vocab)


brown_raw=brown.paras()

#collect all tokens in brown corpus, and store them in a list
tokens=[]
for par in brown_raw:
    for sen in par:
        for word in sen:
            tokens.append(word.lower())


#get the word types in training set
for par in brown_raw[0:num_train]:
    for sen in par:
        for word in sen:
            if word.lower() not in v:
                v.append(word.lower())

#remove word types that has a frequency less than 5
for ele in v:
    
    if tokens.count(ele)>=5:
        vocab.add(ele)




###
# Your answer ENDS HERE
###

print(len(vocab))

14037


**For your testing:**

In [0]:
assert(len(vocab) > 8000 and len(vocab) < 20000)

### Question 7 (1.0 mark)

**Instructions**: As we'll be building a trigram neural language model (based on lecture 7, page 20), the next step is to collect trigrams to construct our training data. In a trigram neural language model, for example if we have the trigram _cow eats grass_, the input to the model is the first two terms of a trigram (_cow_ and _eats_), and the language model's aim is to predict the last term of the trigram (_grass_). Your task here is to construct the training and development data for the language model. Just like the previous step, the first 12K paragraphs will serve as our training data, and the remaining 3K+ will be for development. You'll need to map words into IDs when constructing the training and development data. Any words that are not in _vocab_ should be mapped to the special _$<$UNK$>$_ symbol.

As an example, given the sentence "_a big fat hungry cow ._", you should create the following training examples:

|input|target|
|:---:|:----:|
|<i>a</i>, _big_|_fat_|
|_big_, _fat_|_hungry_|
|_fat_, _hungry_|_cow_|
|_hungry_, _cow_|_._|


Note:
   - _vocab_ is a set and so does not map words into IDs. You'll need to create a word to ID mapping first based on _vocab_.
   - A trigram should not cross sentence boundary.
   - You should ignore sentences that have less than 3 words (as they are too short to form trigrams).
   - We won't need special starting symbol when collecting the trigrams, as we are only interested in learning word embeddings here (and not computing probabilities of a sentence).


**Task**: Create training and development data. The training input and target data should be stored in the <i>x_train</i> and <i>y_train</i> respectively, and development input and target data in <i>x_dev</i> and <i>y_dev</i> respectively.

**Check**: Use the assertion statements in *"For your testing"* below for the expected shape of the outputs.

In [13]:
import numpy as np
###
# Your answer BEGINS HERE
###

#map the word into ID
def mapID(word):
    global vocab
    global UNK_symbol
    #change set into a list
    vlist=list(vocab)
    #map the existed word into the index
    #map the non-existed word into <UNK> index
    if word.lower() in vlist:
        return vlist.index(word.lower())
    else:
        return vlist.index(UNK_symbol)

x_train=[]
y_train=[]
x_dev=[]
y_dev=[]
#create training and development data
i=0
for par in brown_raw:
    for sen in par:
        for j in range(0,len(sen)-2):
            #among the 3 words, the first 2 words are inputs, and the remaining 1 word is target word
            input0=mapID(sen[j])
            input1=mapID(sen[j+1])
            target=mapID(sen[j+2])
            #judge whether to put them in training set or testing set by i
            if i<num_train:
                x_train.append([input0,input1])
                y_train.append(target)
            else:
                x_dev.append([input0,input1])
                y_dev.append(target)
    i=i+1
x_train=np.array(x_train)
x_dev=np.array(x_dev)
y_train=np.array(y_train)
y_dev=np.array(y_dev)






###
# Your answer ENDS HERE
###

    
print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

(872823, 2)
(872823,)
(174016, 2)
(174016,)


**For your testing:**

In [0]:
assert(x_train.shape[0] == y_train.shape[0])
assert(x_dev.shape[0] == y_dev.shape[0])
assert(x_train.shape[0] > 500000)
assert(x_dev.shape[0] > 50000)

### Question 8 (1.0 mark)

**Instructions**: Now let's build the trigram neural language model. We'll use the language model described in lecture 7 (page 20):

$x' = e(x_1) \oplus e(x_2)$

$h = \tanh(W_1 x' + b)$

$y = $ softmax$(W_2 h)$

where $\oplus$ is the concatenation operation, $x_1$ and $x_2$ are the input words, $e$ is an embedding function, and $y$ is the target word. You can use either `keras` or `pytorch` for building your model.

Set the dimension of the word embeddings and $h$ to 100, and train your model with 3 epochs with a batch size of 256. You will not need to tune hyper-parameters for this task.

After the model is trained, use the word embeddings to compute word similarity in the test set. Store the similarity scores in a dictionary called *lm_similarities*.

Note:
  - For words in the test set that are not in your vocabulary, you should treat them as unknown words. In other words, you should use $<$UNK$>$'s embedding to represent these words.
  - The training may take some time on CPU. You can run your notebook on colab with a GPU if you want faster training (see the programming exercise in workshop-07 if you're not familiar with colab)

**Task**: Train a trigram neural language model, and use the learned word embeddings to compute word similarity. Produce *lm_similarities*, a dictionary that contains word pairs as keys and similarity scores as values.

In [31]:

import math
import torch
from torch import nn, optim



lm_similarities = {}

###
# Your answer BEGINS HERE
###


#define a class to generate n-gram model
class n_gram(nn.Module):
    #constructor
    def __init__(self, v_len, n, emb_num):
        super(n_gram, self).__init__()
        #embedding layer
        self.embed = nn.Embedding(v_len, emb_num)
        #classification layer
        self.classify = nn.Sequential(
            nn.Linear(n * emb_num, 100),   
            nn.Tanh(),
            nn.Linear(100, v_len)
        )
    #feedforward function
    def forward(self, x):
        #get the word embeddings
        w_embed = self.embed(x)  
        #concantenate the two vectors
        w_embed = w_embed.view(1, -1)  
        #feed it to the classification layer
        res = self.classify(w_embed)   
        #get the result
        return res
    #calculate the similarities between two words by word embedding vectors
    def get_sim(self,word):
        #get word embeddings
        w_emb=self.embed(word)
        #change it into array
        arr=w_emb.detach().numpy()
        #get the first and second vectors
        l0=arr[0]
        l1=arr[1]
        #calculate the cosine similarities
        sim=cos_sim(l0,l1)
        #return the result
        return sim

#generate the model
model = n_gram(len(vocab), 2, 100)
#define the loss function
loss_func = nn.CrossEntropyLoss()
#define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.00001)
#iterate over 3 epochs
for epoch in range(0,3):
    #store the loss
    train_loss=0
    #iterate by batch size
    for i in range(0,math.ceil(len(x_train)/256)):
        #define the beginning and ending index
        begin=i*256
        end=(i+1)*256
        if end>len(x_train):
            end=len(x_train)
        #generate the training data and their results
        w0arr=x_train[begin:end,0]
        w1arr=x_train[begin:end,1]
        rarr=y_train[begin:end]
        #feed them one by one to the neural networks
        for j in range(0,len(w0arr)):
            #gernerate word index from array
            word0=w0arr[j]
            word1=w1arr[j]
            res=rarr[j]
            #generate tensor from indexes
            word = torch.LongTensor([word0,word1])
            
            result=torch.LongTensor([res])
            #calculate the estimated result
            est = model(word)
            #compare it with the ground truth
            loss = loss_func(est, result)
            train_loss += loss.item()
            #backpropogation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

#generate the test dataset (word indexes)
x_test=[]
templist_test=list(final_gold_standard.keys())
for key in templist_test:
    x_test.append([mapID(key[0]),mapID(key[1])])
x_test=np.array(x_test)


for i in range(0,len(x_test)):
    #get the word indexes
    word0=x_test[i,0]
    word1=x_test[i,1]
    #generate into tensor
    word = torch.LongTensor([word0,word1])
    #calculate the similarities
    sim_value=model.get_sim(word)
    #store the similarities
    lm_similarities[templist_test[i]]=sim_value





###
# Your answer ENDS HERE
###


print(lm_similarities)

{('bread', 'butter'): -0.18302849, ('professor', 'doctor'): -0.12192361, ('student', 'professor'): -0.0347743, ('stock', 'egg'): -0.02946542, ('money', 'cash'): 0.006176472, ('king', 'queen'): -0.021939944, ('bishop', 'rabbi'): 0.049883816, ('football', 'basketball'): -0.011249224, ('football', 'tennis'): 0.13576242, ('alcohol', 'chemistry'): 0.061158713, ('baby', 'mother'): -0.15232055, ('car', 'automobile'): -0.041874226, ('journey', 'voyage'): -0.13344817, ('coast', 'shore'): -0.050318457, ('brother', 'monk'): 0.12634884, ('journey', 'car'): -0.1332163, ('coast', 'hill'): 0.089358956, ('forest', 'graveyard'): -0.06495038, ('monk', 'slave'): 0.1731685, ('coast', 'forest'): -0.09751066, ('psychology', 'doctor'): -0.07891187, ('psychology', 'mind'): 0.064729154, ('psychology', 'health'): 0.04855015, ('psychology', 'science'): 0.21594372, ('planet', 'moon'): 0.015948562, ('planet', 'galaxy'): -0.065028876}


## 3. Comparison with the Gold Standard (1 mark)

### Question 9 (1.0 mark)

**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA', 'lm').

**Task**: Compute the Pearson correlation coefficient between the estimated similarity scores (Lin, NPMI and LSA similarities) and the gold standard similarity ratings. Produce *pearson_correlations*, a dictionary containing the methods as keys and correlations as values.

**Check**: Use the assertion statements in *"For your testing"* below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs! 

In [35]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA', 'lm'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

#change the dictionary into lists
linlist=[]
NPMIlist=[]
LSAlist=[]
lmlist=[]
truelist=[]
for key,value in final_gold_standard.items():
    linlist.append(lin_similarities[key])
    NPMIlist.append(NPMI_similarities[key])
    LSAlist.append(LSA_similarities[key])
    lmlist.append(lm_similarities[key])
    truelist.append(value)
#compare the estimated similarities with ground truth
pearson_correlations['lin'],_=pearsonr(linlist,truelist)
pearson_correlations['NPMI'],_=pearsonr(NPMIlist,truelist)
pearson_correlations['LSA'],_=pearsonr(LSAlist,truelist)
pearson_correlations['lm'],_=pearsonr(lmlist,truelist)    
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.5301489978447533, 'NPMI': 0.19362833174466434, 'LSA': 0.41702544529412355, 'lm': -0.18669149102396046}


<b>For your testing:</b>

In [0]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.